# EventVestor: Issue Debt

In this notebook, we'll take a look at EventVestor's *Issue Debt* dataset, available on the [Quantopian Store](https://www.quantopian.com/store). This dataset spans January 01, 2007 through the current day, and documents events and announcements covering new debt issues by companies.

### Blaze
Before we dig into the data, we want to tell you about how  you generally access Quantopian Store data sets. These datasets are available through an API service known as [Blaze](http://blaze.pydata.org). Blaze provides the Quantopian user with a convenient interface to access very large datasets.

Blaze provides an important function for accessing these datasets. Some of these sets are many millions of records. Bringing that data directly into Quantopian Research directly just is not viable. So Blaze allows us to provide a simple querying interface and shift the burden over to the server side.

It is common to use Blaze to reduce your dataset in size, convert it over to Pandas and then to use Pandas for further computation, manipulation and visualization.

Helpful links:
* [Query building for Blaze](http://blaze.pydata.org/en/latest/queries.html)
* [Pandas-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-pandas.html)
* [SQL-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-sql.html).

Once you've limited the size of your Blaze object, you can convert it to a Pandas DataFrames using:
> `from odo import odo`  
> `odo(expr, pandas.DataFrame)`

### Free samples and limits
One other key caveat: we limit the number of results returned from any given expression to 10,000 to protect against runaway memory usage. To be clear, you have access to all the data server side. We are limiting the size of the responses back from Blaze.

There is a *free* version of this dataset as well as a paid one. The free one includes about three years of historical data, though not up to the current day.

With preamble in place, let's get started:

In [1]:
# import the dataset
from quantopian.interactive.data.eventvestor import issue_debt
# or if you want to import the free dataset, use:
# from quantopian.interactive.data.eventvestor import issue_debt_free

# import data operations
from odo import odo
# import other libraries we will use
import pandas as pd

In [2]:
# Let's use blaze to understand the data a bit using Blaze dshape()
issue_debt.dshape

dshape("""var * {
  event_id: ?float64,
  asof_date: datetime,
  trade_date: ?datetime,
  symbol: ?string,
  event_type: ?string,
  event_headline: ?string,
  issue_amount: ?float64,
  issue_units: ?string,
  issue_stage: ?string,
  event_rating: ?float64,
  timestamp: datetime,
  sid: ?int64
  }""")

In [3]:
# And how many rows are there?
# N.B. we're using a Blaze function to do this, not len()
issue_debt.count()

13844

In [4]:
# Let's see what the data looks like. We'll grab the first three rows.
issue_debt[:3]

,event_id,asof_date,trade_date,symbol,event_type,event_headline,issue_amount,issue_units,issue_stage,event_rating,timestamp,sid
0,932000,2007-01-02,2007-01-02,NMRX,Issue Debt,Numerex to Issue $10M Debt in Private Placement,10,$M,NaN,1,2007-01-03,11002
1,149958,2007-01-02,2007-01-03,TECD,Issue Debt,Tech Data Completes Sale Of $25M Senior Debent...,25,$M,NaN,1,2007-01-03,7372
2,134612,2007-01-04,2007-01-04,RRD,Issue Debt,R.R. Donnelley Prices $1.25B Debt Offering,1250,$M,NaN,1,2007-01-05,2248


Let's go over the columns:
- **event_id**: the unique identifier for this event.
- **asof_date**: EventVestor's timestamp of event capture.
- **trade_date**: for event announcements made before trading ends, trade_date is the same as event_date. For announcements issued after market close, trade_date is next market open day.
- **symbol**: stock ticker symbol of the affected company.
- **event_type**: this should always be *Issue Debt*.
- **event_headline**: a brief description of the event
- **issue_amount**: value of the debt issued in issue_units 
- **issue_units**: units of the issue_amount, most commonly millions of dollars.
- **issue_stage**: phase of the issue process: announcement, closing, pricing, etc. Note: currently, there appear to be unrelated entries in this column. We are speaking with the data vendor to amend this.
- **event_rating**: this is always 1. The meaning of this is uncertain.
- **timestamp**: this is our timestamp on when we registered the data.
- **sid**: the equity's unique identifier. Use this instead of the symbol.

We've done much of the data processing for you. Fields like `timestamp` and `sid` are standardized across all our Store Datasets, so the datasets are easy to combine. We have standardized the `sid` across all our equity databases.

We can select columns and rows with ease. Below, we'll fetch all 2015 debt issues smaller than $20M.

In [5]:
issues = issue_debt[('2014-12-31' < issue_debt['asof_date']) & 
                                        (issue_debt['asof_date'] <'2016-01-01') & 
                                        (issue_debt.issue_amount < 20)&
                                        (issue_debt.issue_units  == "$M")]
# When displaying a Blaze Data Object, the printout is automatically truncated to ten rows.
issues.sort('asof_date')

,event_id,asof_date,trade_date,symbol,event_type,event_headline,issue_amount,issue_units,issue_stage,event_rating,timestamp,sid
0,1820589,2015-01-06,2015-01-07,MVC,Issue Debt,MVC Capital Draws $15.9M from Credit Facility,15.900,$M,Announcement,1,2015-01-07 00:00:00,21667
1,1821690,2015-01-09,2015-01-09,MNTX,Issue Debt,Manitex International Issues $15M Notes,15.000,$M,Announcement,1,2015-01-10 00:00:00,27042
2,1823391,2015-01-14,2015-01-15,TAT,Issue Debt,TransAtlantic Petroluem Issues Additional $1.3...,1.300,$M,NaN,1,2015-01-15 00:00:00,38304
3,1830977,2015-02-03,2015-02-03,EBTC,Issue Debt,Enterprise Bancorp Issues $15M Notes Due 2030,15.000,$M,Announcement,1,2015-02-04 00:00:00,27032
4,1859691,2015-02-10,2015-02-11,TAT,Issue Debt,TransAtlantic Petroluem Sells Additional $1M C...,1.000,$M,Announcement,1,2015-02-11 00:00:00,38304
5,1860242,2015-02-12,2015-02-12,T,Issue Debt,AT&T Issues $2.62B Notes Due 2045,2.619,$M,Announcement,1,2015-02-13 00:00:00,6653
6,1860506,2015-02-20,2015-02-23,VGGL,Issue Debt,Viggle Gets $0.75M Unsecured Demand Loan from ...,0.750,$M,Announcement,1,2015-02-21 00:00:00,31350
7,1852384,2015-03-03,2015-03-03,HPT,Issue Debt,Mobi724 Global Solutions Completes Third Tranc...,0.120,$M,NaN,1,2015-03-04 00:00:00,13373
8,1845300,2015-03-03,2015-03-03,CNDO,Issue Debt,Coronado Biosciences Closes $10M Notes Offering,10.000,$M,Announcement,1,2015-03-04 00:00:00,NaN
9,1845300,2015-03-03,2015-03-03,CNDO,Issue Debt,Coronado Biosciences Closes $10M Notes Offering,10.000,$M,Announcement,1,2015-09-29 10:53:30.635231,NaN


Now suppose we want a DataFrame of the Blaze Data Object above, want to filter it further down to the announcements only, and we only want the sid, issue_amount, and the asof_date.

In [6]:
df = odo(issues, pd.DataFrame)
df = df[df.issue_stage == "Announcement"]
df = df[['sid', 'issue_amount', 'asof_date']].dropna()
df

,sid,issue_amount,asof_date
0,21667,15.9000,2015-01-06
1,27042,15.0000,2015-01-09
3,27032,15.0000,2015-02-03
4,38304,1.0000,2015-02-10
5,6653,2.6190,2015-02-12
6,31350,0.7500,2015-02-20
10,8151,0.7900,2015-03-04
11,8151,9.9540,2015-03-04
12,26723,9.3200,2015-03-10
14,8151,6.9710,2015-03-27
